In [330]:
from sympy import Symbol, Matrix, Eq, MatAdd, MatMul, preview, S, zeros, solve


def get_symbols_dict(s: str) -> dict:

    return {i: Symbol(i) for i in s.split(' ')}


def create_endog_v(endog: dict) -> Matrix:
    """
    Вектор для эндогенных
    """
    return Matrix([[i] for i in endog.values()])


def create_predifined_v(pseudos: list, predeters: dict) -> Matrix:
    """
    Вектор для предопределенных
    """
    return Matrix([[i] for i in [*pseudos, *predeters.values()]])


def calc_M(A: Matrix, B: Matrix) -> Matrix:
    """
    M = -A ** (-1) * B
    """
    return -A.inv() * B


def get_reduced_form(X: Matrix, Y: Matrix, A: Matrix, M: Matrix, U: Matrix) -> Eq:
    """
    Получаем приведенную матрицу
    """
    if zeros(*U.shape) == U:
        res = MatMul(M, X)
    else:
        res = MatAdd(MatMul(M, X), MatMul(A.inv(), U))
    return Eq(Y, res, evaluate=False)


## 1.4 Модель корректировки размера дивидендов Линтера

Задача 4. Модель корректировки размера дивидендов Линтнера.<br>Пусть $\pi_{t}$ - текущая прибыль фирмы после уплаты налогов,<br>$D_{t}$ - дивиденды, которые фирма выплачивает своим акционерам в текущем периоде. Известный американский экономист Дж. Линтнер сформулировал в 1956г. следующие утверждения:
1) у фирмы имеется долгосрочная целевая доля $g$ текущей прибыли и соответствующий этой доле уровень дивидендов $D_{t}^{*}$, которые фирма хотела бы выплачивать своим акционерам (желаемый уровень);<br>
2) Изменение $\left(D_{t}-D_{t-1}\right)$ уровня реальных дивидендов за период прямо пропорционально отличию желаемого объёма $D_{t}^{*}$ текущих дивидендов от их реального уровня в предшествующем периоде $D_{t-1} .$<br>Требуется:
a) составить спецификацию модели корректировки размера дивидендов, позволяющую объяснить текущий уровень $D_{t}$ дивидендов их лаговым значением и текущей прибылью фирмы;<br>
б) Уточнить спецификацию включением случайного возмущения.<br>
в) Представить структурную форму спецификации в матричном виде.<br>
г) Составить приведенную форму модели.<br>
д) Записать приведенную форму в матричном виде.<br>

#### Дано

In [331]:
endog = 'D_t D^*_t'
endog = get_symbols_dict(endog)

predifins = 'P_t D_t-1'
predifins = get_symbols_dict(predifins)

params = 'g a b'
params = get_symbols_dict(params)

mdevs = 'ε0'
mdevs = get_symbols_dict(mdevs)

# Система

eq_system = [
    Eq(endog['D^*_t'],
       params['g'] * predifins['P_t']),
    Eq(endog['D_t'] - predifins['D_t-1'],
       params['a'] + params['b'] * (endog['D^*_t'] - predifins['D_t-1']) + mdevs['ε0']),
]

# Приведенная

reduced_form_system = solve(eq_system, *endog.values())


Y = create_endog_v(endog)
X = create_predifined_v([1], predifins)

# матрица коэфф Эндогенных
A = Matrix([
    [0, 1],
    [1, -params['b']],
])

# матрица коэфф Предопределенных
B = Matrix([
    [0, -params['g'], 0],
    [-params['a'], 0, params['b'] - 1],
])

U = Matrix([
    [0],
    [mdevs['ε0']],
])


M = calc_M(A, B)
reduced_form_matrix = get_reduced_form(X, Y, A, M, U)


In [332]:
Eq(Symbol('Y'), Y, evaluate=False)

Eq(Y, Matrix([
[  D_t],
[D^*_t]]))

In [333]:
Eq(Symbol('X'), X, evaluate=False)

Eq(X, Matrix([
[    1],
[  P_t],
[D_t-1]]))

In [334]:
Eq(Symbol('A'), A, evaluate=False)

Eq(A, Matrix([
[0,  1],
[1, -b]]))

In [335]:
Eq(Symbol('B'), B, evaluate=False)

Eq(B, Matrix([
[ 0, -g,     0],
[-a,  0, b - 1]]))

In [336]:
Eq(Symbol('U'), U, evaluate=False)

Eq(U, Matrix([
[ 0],
[ε0]]))

### Приведенная форма (матричная)

In [337]:
reduced_form_matrix

Eq(Matrix([
[  D_t],
[D^*_t]]), Matrix([
[b, 1],
[1, 0]])*Matrix([
[ 0],
[ε0]]) + Matrix([
[a, b*g, 1 - b],
[0,   g,     0]])*Matrix([
[    1],
[  P_t],
[D_t-1]]))

### Приведенная форма (система)

In [338]:
Eq(Symbol('D_t'), reduced_form_system[Symbol('D_t')], evaluate=False)

Eq(D_t, -D_t-1*b + D_t-1 + P_t*b*g + a + ε0)

In [339]:
Eq(Symbol('D^*_t'), reduced_form_system[Symbol('D^*_t')], evaluate=False)

Eq(D^*_t, P_t*g)

## 2.1 Модель Манделла - Флеминга (IS-LM-BP)

2.1. Необходимо составить спецификацию модификации модели IS - $L M$ - модели Манделла-Флеминга открытой экономики в краткосрочном периоде с плавающим обменным курсом.

Состояние национальной экономики в заданном периоде времени описывается количественными характеристиками $Y, C, I, N X, E, R, L$, $C F, M, P, G, T$

где Y - объем (величина) выпуска (совокупный спрос, валовый внутренний продукт);<br>
С - уровень потребления;<br>
I - объем инвестиций в экономику;<br>
R - ставка процента (рефинансирования);<br>
L - спрос на деньги в реальном выражении (не путать с уровнем затрат труда!);<br>
М - уровень предложения денег;<br>
Р - уровень цен;<br>
G - объем государственных расходов;<br>
Т - уровень налогов.<br>
Используя приведенные ниже утверждения, составьте спецификацию макромодели Манделла—Флеминга, предназначенной для объяснения переменных Y, R ,L, I, С, NX, Е и CF значениями переменных М,Р, G и Т.<br>

При составлении спецификации модели следует использовать следующие утверждения:
1.	Объем выпуска Y является суммой частного потребления С, инвестиций I, государственного потребления (расходов) G и чистого экспорта
NX;
2.	Уровень потребления С объясняется располагаемым доходом (Y — Т), и возрастает в ответ на увеличение этого дохода, при этом каждая дополнительная единица располагаемого дохода потребляется не полностью;
3.	Объем инвестиций в экономику / объясняется реальной ставкой процента R и снижается в ответ на рост этой ставки;
4.	Уровень спроса на деньги в реальном выражении L объясняется реальной ставкой процента R и величиной дохода Y; L снижается при увеличении R и возрастает в ответ на увеличение Y;
5.	На денежном рынке существует равновесие спроса и предложения денег в реальном выражении;
6.	Уровень чистого экспорта NX объясняется номинальным обменным курсом Е, ив ответ на увеличение номинального обменного курса уровень чистого экспорта снижается;
7.	Приток иностранного капитала CF (счет движения капитала платежного баланса) объясняется реальной ставкой процента R и увеличивается в ответ на рост этой ставки;
8.	Счет движения капитала платежного баланса CF и счет текущих операций NX уравновешивают друг друга, т.е. в сумме равны нулю.
После составления спецификации модели Манделла-Флеминга преобразуйте ее к приведенной форме. Запишите модель в матричном виде, указав вектора эндогенных и экзогенных векторов, матрицы коэффициентов при эндогенных и экзогенных переменных. Получив приведенную модель запишите результирующую матрицу


In [340]:
"""
    1) Y = C + I + G + NX
    2) C = a + b * (Y-T) + ε0; 0 < b < 1
    3) I = c + d * R + ε1; d < 0
    4) L = f + g * R + h * Y + ε2, g < 0, h > 0
    5) L = M
    6) NX = i + j * E + ε3, j < 0
    7) CF = k + n * R + ε4, n > 0
    8) CF + NX = 0
"""

endog = 'Y C I NX E R L CF'
endog = get_symbols_dict(endog)

predifins = 'M P G T'
predifins = get_symbols_dict(predifins)

params = 'a b c d f g h i j k n'
params = get_symbols_dict(params)

mdevs = 'ε0 ε1 ε2 ε3 ε4'
mdevs = get_symbols_dict(mdevs)

# Система

eq_system = [
    Eq(endog['Y'],
       endog['C'] + endog['I'] + predifins['G'] + endog['NX']),
    Eq(endog['C'],
       params['a'] + params['b'] * (endog['Y'] - predifins['T']) + mdevs['ε0']),
    Eq(endog['I'],
       params['c'] + params['d'] * endog['R'] + mdevs['ε1']),
    Eq(endog['L'],
       params['f'] + params['g'] * endog['Y'] + params['h'] * endog['R'] + mdevs['ε2']),
    Eq(predifins['M'],
       endog['L']),
    Eq(endog['NX'],
       params['i'] + params['j'] * endog['E'] + mdevs['ε3']),
    Eq(endog['CF'],
       params['k'] + params['n'] * endog['R'] + mdevs['ε4']),
    Eq(0,
       endog['CF'] + endog['NX']),
]

# Приведенная

reduced_form_system = solve(eq_system, *endog.values())

Y = create_endog_v(endog)
X = create_predifined_v([1], predifins)

# матрица коэфф Эндогенных

A = Matrix([
    [1, -1, -1, -1, 0, 0, 0, 0],
    [-params['b'], 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, -params['d'], 0, 0],
    [-params['g'], 0, 0, 0, 0, -params['h'], 1, 0],
    [0, 0, 0, 0, 0, 0, -1, 0],
    [0, 0, 0, 1, -params['j'], 0, 0, 0],
    [0, 0, 0, 0, 0, -params['n'], 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 1],
])

# матрица коэфф Предопределенных
B = Matrix([
    [0, 0, 0, -1, 0],
    [-params['a'], 0, 0, 0, params['b']],
    [-params['c'], 0, 0, 0, 0],
    [-params['f'], 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [-params['i'], 0, 0, 0, 0],
    [-params['k'], 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
])

U = Matrix([
    [0],
    [mdevs['ε0']],
    [mdevs['ε1']],
    [mdevs['ε2']],
    [0],
    [mdevs['ε3']],
    [mdevs['ε4']],
    [0],
])


M = calc_M(A, B)
reduced_form_matrix = get_reduced_form(X, Y, A, M, U)


In [341]:
Eq(Symbol('Y'), Y, evaluate=False)

Eq(Y, Matrix([
[ Y],
[ C],
[ I],
[NX],
[ E],
[ R],
[ L],
[CF]]))

In [342]:
Eq(Symbol('X'), X, evaluate=False)

Eq(X, Matrix([
[1],
[M],
[P],
[G],
[T]]))

In [343]:
Eq(Symbol('A'), A, evaluate=False)

Eq(A, Matrix([
[ 1, -1, -1, -1,  0,  0,  0, 0],
[-b,  1,  0,  0,  0,  0,  0, 0],
[ 0,  0,  1,  0,  0, -d,  0, 0],
[-g,  0,  0,  0,  0, -h,  1, 0],
[ 0,  0,  0,  0,  0,  0, -1, 0],
[ 0,  0,  0,  1, -j,  0,  0, 0],
[ 0,  0,  0,  0,  0, -n,  0, 1],
[ 0,  0,  0,  1,  0,  0,  0, 1]]))

In [344]:
Eq(Symbol('B'), B, evaluate=False)

Eq(B, Matrix([
[ 0, 0, 0, -1, 0],
[-a, 0, 0,  0, b],
[-c, 0, 0,  0, 0],
[-f, 0, 0,  0, 0],
[ 0, 1, 0,  0, 0],
[-i, 0, 0,  0, 0],
[-k, 0, 0,  0, 0],
[ 0, 0, 0,  0, 0]]))

In [345]:
Eq(Symbol('U'), U, evaluate=False)

Eq(U, Matrix([
[ 0],
[ε0],
[ε1],
[ε2],
[ 0],
[ε3],
[ε4],
[ 0]]))

### Приведенная форма (матричная)

In [346]:
reduced_form_matrix

Eq(Matrix([
[ Y],
[ C],
[ I],
[NX],
[ E],
[ R],
[ L],
[CF]]), Matrix([
[                                                           -a*h/(b*h - d*g + g*n - h) - c*h/(b*h - d*g + g*n - h) + f*(d - n)/(b*h - d*g + g*n - h) + h*k/(b*h - d*g + g*n - h),           -(d - n)/(b*h - d*g + g*n - h), 0,           -h/(b*h - d*g + g*n - h),                 b*h/(b*h - d*g + g*n - h)],
[                                     a*(-d*g + g*n - h)/(b*h - d*g + g*n - h) - b*c*h/(b*h - d*g + g*n - h) + b*h*k/(b*h - d*g + g*n - h) + f*(b*d - b*n)/(b*h - d*g + g*n - h),       -(b*d - b*n)/(b*h - d*g + g*n - h), 0,         -b*h/(b*h - d*g + g*n - h), -b*(-d*g + g*n - h)/(b*h - d*g + g*n - h)],
[                                       a*d*g/(b*h - d*g + g*n - h) + c*(b*h + g*n - h)/(b*h - d*g + g*n - h) - d*g*k/(b*h - d*g + g*n - h) + f*(-b*d + d)/(b*h - d*g + g*n - h),        -(-b*d + d)/(b*h - d*g + g*n - h), 0,          d*g/(b*h - d*g + g*n - h),              -b*d*g/(b*h - d*g + g*n - h)],
[                   

### Приведенная форма (система)

In [347]:
Eq(Symbol('Y'), reduced_form_system[Symbol('Y')], evaluate=False)

Eq(Y, (-G*h - M*d + M*n + T*b*h - a*h - c*h + d*f + d*ε2 - f*n + h*k - h*ε0 - h*ε1 + h*ε4 - n*ε2)/(b*h - d*g + g*n - h))

In [348]:
Eq(Symbol('С'), reduced_form_system[Symbol('C')], evaluate=False)

Eq(С, (-G*b*h - M*b*d + M*b*n + T*b*d*g - T*b*g*n + T*b*h - a*d*g + a*g*n - a*h - b*c*h + b*d*f + b*d*ε2 - b*f*n + b*h*k - b*h*ε1 + b*h*ε4 - b*n*ε2 - d*g*ε0 + g*n*ε0 - h*ε0)/(b*h - d*g + g*n - h))

In [349]:
Eq(Symbol('I'), reduced_form_system[Symbol('I')], evaluate=False)

Eq(I, (G*d*g + M*b*d - M*d - T*b*d*g + a*d*g + b*c*h - b*d*f - b*d*ε2 + b*h*ε1 + c*g*n - c*h + d*f - d*g*k + d*g*ε0 - d*g*ε4 + d*ε2 + g*n*ε1 - h*ε1)/(b*h - d*g + g*n - h))

In [350]:
Eq(Symbol('NX'), reduced_form_system[Symbol('NX')], evaluate=False)

Eq(NX, (-G*g*n - M*b*n + M*n + T*b*g*n - a*g*n + b*f*n - b*h*k - b*h*ε4 + b*n*ε2 - c*g*n + d*g*k + d*g*ε4 - f*n - g*n*ε0 - g*n*ε1 + h*k + h*ε4 - n*ε2)/(b*h - d*g + g*n - h))

In [351]:
Eq(Symbol('E'), reduced_form_system[Symbol('E')], evaluate=False)

Eq(E, (-G*g*n - M*b*n + M*n + T*b*g*n - a*g*n + b*f*n - b*h*i - b*h*k - b*h*ε3 - b*h*ε4 + b*n*ε2 - c*g*n + d*g*i + d*g*k + d*g*ε3 + d*g*ε4 - f*n - g*i*n - g*n*ε0 - g*n*ε1 - g*n*ε3 + h*i + h*k + h*ε3 + h*ε4 - n*ε2)/(b*h*j - d*g*j + g*j*n - h*j))

In [352]:
Eq(Symbol('R'), reduced_form_system[Symbol('R')], evaluate=False)

Eq(R, (G*g + M*b - M - T*b*g + a*g - b*f - b*ε2 + c*g + f - g*k + g*ε0 + g*ε1 - g*ε4 + ε2)/(b*h - d*g + g*n - h))

In [353]:
Eq(Symbol('L'), reduced_form_system[Symbol('L')], evaluate=False)

Eq(L, M)

In [354]:
Eq(Symbol('CF'), reduced_form_system[Symbol('CF')], evaluate=False)

Eq(CF, (G*g*n + M*b*n - M*n - T*b*g*n + a*g*n - b*f*n + b*h*k + b*h*ε4 - b*n*ε2 + c*g*n - d*g*k - d*g*ε4 + f*n + g*n*ε0 + g*n*ε1 - h*k - h*ε4 + n*ε2)/(b*h - d*g + g*n - h))

## 2.2 Модель денежного и товарного рынков

Представленную ниже структурную форму модели преобразуйте к приведенной. Запишите приведенную форму задачи в матричном виде.
Модель денежного и товарного рынков

Модель денежного и товарного рынков:<br>
R = a1 + b12 * Y + b14 * M               (функция денежного рынка)<br>
Y = a2 + b21 * R + b23 * I + b25 * G     (функция товарного рынка)<br>
I = a3 + b31 * R                         (функция инвестиций)<br>
<br>R - процентные ставки
<br>Y - реальный ВВП
<br>M - денежная масса
<br>I - внутренние инвестиции
<br>G - реальные государственные расходы

In [355]:
endog = 'R Y I'
endog = get_symbols_dict(endog)

predifins = 'M G'
predifins = get_symbols_dict(predifins)

params = 'a1 a2 a3 b12 b14 b21 b23 b25 b31'
params = get_symbols_dict(params)

# Система

eq_system = [
    Eq(endog['R'],
       params['a1'] + params['b12'] * endog['Y'] + params['b14'] * predifins['M']),
    Eq(endog['Y'],
       params['a2'] + params['b21'] * endog['R'] + params['b23'] * endog['I'] + params['b25'] * predifins['G']),
    Eq(endog['I'],
       params['a3'] + params['b31'] * endog['R']),
]

# Приведенная

reduced_form_system = solve(eq_system, *endog.values())


Y = create_endog_v(endog)
X = create_predifined_v([1], predifins)

# матрица коэфф Эндогенных
A = Matrix([
    [1, -params['b12'], 0],
    [-params['b21'], 1, -params['b23']],
    [-params['b31'], 0, 1]
])

# матрица коэфф Предопределенных
B = Matrix([
    [-params['a1'], -params['b14'], 0],
    [-params['a2'], 0, -params['b25']],
    [-params['a3'], 0, 0],
])

U = Matrix([
    [0],
    [0],
    [0]
])


M = calc_M(A, B)
reduced_form_matrix = get_reduced_form(X, Y, A, M, U)

In [356]:
Eq(Symbol('Y'), Y, evaluate=False)

Eq(Y, Matrix([
[R],
[Y],
[I]]))

In [357]:
Eq(Symbol('X'), X, evaluate=False)

Eq(X, Matrix([
[1],
[M],
[G]]))

In [358]:
Eq(Symbol('A'), A, evaluate=False)

Eq(A, Matrix([
[   1, -b12,    0],
[-b21,    1, -b23],
[-b31,    0,    1]]))

In [359]:
Eq(Symbol('B'), B, evaluate=False)

Eq(B, Matrix([
[-a1, -b14,    0],
[-a2,    0, -b25],
[-a3,    0,    0]]))

In [360]:
Eq(Symbol('U'), U, evaluate=False)

Eq(U, Matrix([
[0],
[0],
[0]]))

### Приведенная форма (матричная)

In [361]:
reduced_form_matrix

Eq(Matrix([
[R],
[Y],
[I]]), Matrix([
[              -a1/(b12*b21 + b12*b23*b31 - 1) - a2*b12/(b12*b21 + b12*b23*b31 - 1) - a3*b12*b23/(b12*b21 + b12*b23*b31 - 1),                 -b14/(b12*b21 + b12*b23*b31 - 1),     -b12*b25/(b12*b21 + b12*b23*b31 - 1)],
[      a1*(-b21 - b23*b31)/(b12*b21 + b12*b23*b31 - 1) - a2/(b12*b21 + b12*b23*b31 - 1) - a3*b23/(b12*b21 + b12*b23*b31 - 1), b14*(-b21 - b23*b31)/(b12*b21 + b12*b23*b31 - 1),         -b25/(b12*b21 + b12*b23*b31 - 1)],
[-a1*b31/(b12*b21 + b12*b23*b31 - 1) - a2*b12*b31/(b12*b21 + b12*b23*b31 - 1) + a3*(b12*b21 - 1)/(b12*b21 + b12*b23*b31 - 1),             -b14*b31/(b12*b21 + b12*b23*b31 - 1), -b12*b25*b31/(b12*b21 + b12*b23*b31 - 1)]])*Matrix([
[1],
[M],
[G]]))

### Приведенная форма (система)

In [362]:
Eq(Symbol('R'), reduced_form_system[Symbol('R')], evaluate=False)

Eq(R, (-G*b12*b25 - M*b14 - a1 - a2*b12 - a3*b12*b23)/(b12*b21 + b12*b23*b31 - 1))

In [363]:
Eq(Symbol('I'), reduced_form_system[Symbol('I')], evaluate=False)

Eq(I, (-G*b12*b25*b31 - M*b14*b31 - a1*b31 - a2*b12*b31 + a3*b12*b21 - a3)/(b12*b21 + b12*b23*b31 - 1))

In [364]:
Eq(Symbol('Y'), reduced_form_system[Symbol('Y')], evaluate=False)

Eq(Y, (-G*b25 - M*b14*b21 - M*b14*b23*b31 - a1*b21 - a1*b23*b31 - a2 - a3*b23)/(b12*b21 + b12*b23*b31 - 1))

## 2.3 Гипотетическая модель экономики

Представленную ниже структурную форму модели преобразуйте к приведенной. Запишите приведенную форму задачи в матричном виде.

<br>Гипотетическая модель экономики:
<br>C = a1 + b11 * Y + b12 * J
<br>J = a2 + b21 * Y_t-1
<br>T = a3 + b31 * Y
<br>Y = C + J + G
<br><br>C - совокупное потребление в период t
<br>Y - совокупный доход в период t
<br>J - инвестиции в период t
<br>T - налог в период t
<br>G - государственные доходы в период t

In [365]:
endog = 'C Y J T'
endog = get_symbols_dict(endog)

predifins = 'Y_t-1 G'
predifins = get_symbols_dict(predifins)

params = 'a1 a2 a3 b11 b12 b21 b31'
params = get_symbols_dict(params)

# Система

eq_system = [
    Eq(endog['C'],
       params['a1'] + params['b11'] * endog['Y'] + params['b12'] * endog['J']),
    Eq(endog['J'],
       params['a2'] + params['b21'] * predifins['Y_t-1']),
    Eq(endog['T'],
       params['a3'] + params['b31'] * endog['Y']),
    Eq(endog['Y'],
       endog['C'] + endog['J'] + predifins['G']),
]

# Приведенная

reduced_form_system = solve(eq_system, *endog.values())


Y = create_endog_v(endog)
X = create_predifined_v([1], predifins)

# матрица коэфф Эндогенных
A = Matrix([
    [1, -params['b12'], 0, -params['b11']],
    [0, 1, 0, 0],
    [0, 0, 1, -params['b31']],
    [-1, -1, 0, 1],
])

# матрица коэфф Предопределенных
B = Matrix([
    [-params['a1'], 0, 0],
    [-params['a2'], -params['b21'], 0],
    [-params['a3'], 0, 0],
    [0, 0, -1],
])

U = Matrix([
    [0],
    [0],
    [0],
    [0],
])


M = calc_M(A, B)
reduced_form_matrix = get_reduced_form(X, Y, A, M, U)


In [366]:
Eq(Symbol('Y'), Y, evaluate=False)

Eq(Y, Matrix([
[C],
[Y],
[J],
[T]]))

In [367]:
Eq(Symbol('X'), X, evaluate=False)

Eq(X, Matrix([
[    1],
[Y_t-1],
[    G]]))

In [368]:
Eq(Symbol('A'), A, evaluate=False)

Eq(A, Matrix([
[ 1, -b12, 0, -b11],
[ 0,    1, 0,    0],
[ 0,    0, 1, -b31],
[-1,   -1, 0,    1]]))

In [369]:
Eq(Symbol('B'), B, evaluate=False)

Eq(B, Matrix([
[-a1,    0,  0],
[-a2, -b21,  0],
[-a3,    0,  0],
[  0,    0, -1]]))

In [370]:
Eq(Symbol('U'), U, evaluate=False)

Eq(U, Matrix([
[0],
[0],
[0],
[0]]))

### Приведенная форма (матричная)

In [371]:
reduced_form_matrix

Eq(Matrix([
[C],
[Y],
[J],
[T]]), Matrix([
[              a1/(1 - b11) + a2*(b11 + b12)/(1 - b11),           b21*(b11 + b12)/(1 - b11), b11/(1 - b11)],
[                                                   a2,                                 b21,             0],
[a1*b31/(1 - b11) + a2*(-b12*b31 - b31)/(b11 - 1) + a3,      b21*(-b12*b31 - b31)/(b11 - 1), b31/(1 - b11)],
[     a1/(1 - b11) - a2*(-b12/(1 - b11) - 1/(1 - b11)), -b21*(-b12/(1 - b11) - 1/(1 - b11)),   1/(1 - b11)]])*Matrix([
[    1],
[Y_t-1],
[    G]]))

### Приведенная форма (система)

In [372]:
Eq(Symbol('C'), reduced_form_system[Symbol('C')], evaluate=False)

Eq(C, (-G*b11 - Y_t-1*b11*b21 - Y_t-1*b12*b21 - a1 - a2*b11 - a2*b12)/(b11 - 1))

In [373]:
Eq(Symbol('J'), reduced_form_system[Symbol('J')], evaluate=False)

Eq(J, Y_t-1*b21 + a2)

In [374]:
Eq(Symbol('T'), reduced_form_system[Symbol('T')], evaluate=False)

Eq(T, (-G*b31 - Y_t-1*b12*b21*b31 - Y_t-1*b21*b31 - a1*b31 - a2*b12*b31 - a2*b31 + a3*b11 - a3)/(b11 - 1))

In [375]:
Eq(Symbol('Y'), reduced_form_system[Symbol('Y')], evaluate=False)

Eq(Y, (-G - Y_t-1*b12*b21 - Y_t-1*b21 - a1 - a2*b12 - a2)/(b11 - 1))